In [135]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
#col, from_utc_timestamp,upper,lower,initcap,concat_ws,regexp_replace,isnan,when,count,coalesce

In [136]:
spark=SparkSession.builder.master('local[1]').appName('Credit Card Management System').getOrCreate()

### DATA WRANGLING

Cleaning file 'cdw_sapp_custmer.json'

In [137]:
df_cust=spark.read.json('cdw_sapp_custmer.json')
df_cust.show(4)

+------+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|   656|4210653310061055|     Natchez|United States|AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         Wm|123456100|Main Street North|
|   829|4210653310102868|Wethersfield|United States|EHolman@example.com|   1238933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    Brendan|123453023|    Redwood Drive|
|   683|4210653310116272|     Huntley|United States|WDunham@example.co

In [138]:
#checking for duplicates
df_cust.groupBy(df_cust.columns).count().where('count>1').show()
(df_cust.distinct().count())==(df_cust.count())

+------+--------------+---------+------------+----------+----------+----------+--------+----------+---------+------------+-----------+---+-----------+-----+
|APT_NO|CREDIT_CARD_NO|CUST_CITY|CUST_COUNTRY|CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|LAST_UPDATED|MIDDLE_NAME|SSN|STREET_NAME|count|
+------+--------------+---------+------------+----------+----------+----------+--------+----------+---------+------------+-----------+---+-----------+-----+
+------+--------------+---------+------------+----------+----------+----------+--------+----------+---------+------------+-----------+---+-----------+-----+



True

In [139]:
def check_nulls(df):
    navalues=df.select([
    (
        count(when((isnan(c) | col(c).isNull()), c)) if t not in ("timestamp", "date")
        else count(when(col(c).isNull(), c))
    ).alias(c)
    for c, t in df.dtypes if c in df.columns])
    return navalues

check_nulls(df_cust).show()

+------+--------------+---------+------------+----------+----------+----------+--------+----------+---------+------------+-----------+---+-----------+
|APT_NO|CREDIT_CARD_NO|CUST_CITY|CUST_COUNTRY|CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|LAST_UPDATED|MIDDLE_NAME|SSN|STREET_NAME|
+------+--------------+---------+------------+----------+----------+----------+--------+----------+---------+------------+-----------+---+-----------+
|     0|             0|        0|           0|         0|         0|         0|       0|         0|        0|           0|          0|  0|          0|
+------+--------------+---------+------------+----------+----------+----------+--------+----------+---------+------------+-----------+---+-----------+



In [140]:
df_cust=df_cust.withColumn('SSN',col('SSN').cast('int'))
df_cust=df_cust.withColumn('LAST_UPDATED',from_utc_timestamp('LAST_UPDATED','UTC'))
df_cust=df_cust.withColumn('FIRST_NAME',initcap(col('FIRST_NAME')))
df_cust=df_cust.withColumn('MIDDLE_NAME',lower('MIDDLE_NAME'))
df_cust=df_cust.withColumn('LAST_NAME',initcap(col('LAST_NAME')))
df_cust=df_cust.withColumn('FULL_STREET_ADDRESS',concat_ws(',',df_cust.STREET_NAME,df_cust.APT_NO))
df_cust=df_cust.withColumn('CUST_ZIP',col('CUST_ZIP').cast('int'))
df_cust=df_cust.withColumn('CUST_PHONE',regexp_replace(col('CUST_PHONE'),'^(\d{1})(\d{3})(\d{3})$','(+$1)$2-$3'))

df_cust.printSchema()

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: integer (nullable = true)
 |-- STREET_NAME: string (nullable = true)
 |-- FULL_STREET_ADDRESS: string (nullable = false)



In [141]:
df_cust=df_cust.drop(*['APT_NO','STREET_NAME'])

In [142]:
df_cust.columns

['CREDIT_CARD_NO',
 'CUST_CITY',
 'CUST_COUNTRY',
 'CUST_EMAIL',
 'CUST_PHONE',
 'CUST_STATE',
 'CUST_ZIP',
 'FIRST_NAME',
 'LAST_NAME',
 'LAST_UPDATED',
 'MIDDLE_NAME',
 'SSN',
 'FULL_STREET_ADDRESS']

In [143]:
col_order=['SSN','FIRST_NAME','MIDDLE_NAME','LAST_NAME','FULL_STREET_ADDRESS','CUST_CITY','CUST_STATE',
 'CUST_COUNTRY','CUST_ZIP','CUST_PHONE','CUST_EMAIL','LAST_UPDATED']
df_cust=df_cust.select(*col_order)

In [144]:
df_cust.show(truncate=False)

+---------+----------+-----------+---------+---------------------+------------+----------+-------------+--------+-----------+----------------------+-------------------+
|SSN      |FIRST_NAME|MIDDLE_NAME|LAST_NAME|FULL_STREET_ADDRESS  |CUST_CITY   |CUST_STATE|CUST_COUNTRY |CUST_ZIP|CUST_PHONE |CUST_EMAIL            |LAST_UPDATED       |
+---------+----------+-----------+---------+---------------------+------------+----------+-------------+--------+-----------+----------------------+-------------------+
|123456100|Alec      |wm         |Hooper   |Main Street North,656|Natchez     |MS        |United States|39120   |(+1)237-818|AHooper@example.com   |2018-04-21 12:49:02|
|123453023|Etta      |brendan    |Holman   |Redwood Drive,829    |Wethersfield|CT        |United States|6109    |(+1)238-933|EHolman@example.com   |2018-04-21 12:49:02|
|123454487|Wilber    |ezequiel   |Dunham   |12th Street East,683 |Huntley     |IL        |United States|60142   |(+1)243-018|WDunham@example.com   |2018-04

Cleaning file 'cdw_sapp_branch.json'

In [145]:
df_branch=spark.read.json('cdw_sapp_branch.json')
check_nulls(df_branch).show()

+-----------+-----------+-----------+------------+------------+-------------+----------+------------+
|BRANCH_CITY|BRANCH_CODE|BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|BRANCH_STREET|BRANCH_ZIP|LAST_UPDATED|
+-----------+-----------+-----------+------------+------------+-------------+----------+------------+
|          0|          0|          0|           0|           0|            0|         0|           0|
+-----------+-----------+-----------+------------+------------+-------------+----------+------------+



In [146]:
#checking for duplicates
df_branch.groupBy(df_branch.columns).count().where('count>1').show()
(df_branch.distinct().count())==(df_branch.count())

+-----------+-----------+-----------+------------+------------+-------------+----------+------------+-----+
|BRANCH_CITY|BRANCH_CODE|BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|BRANCH_STREET|BRANCH_ZIP|LAST_UPDATED|count|
+-----------+-----------+-----------+------------+------------+-------------+----------+------------+-----+
+-----------+-----------+-----------+------------+------------+-------------+----------+------------+-----+



True

In [147]:
df_branch.show(3)

+-----------------+-----------+------------+------------+------------+-----------------+----------+--------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------------+-----------+------------+------------+------------+-----------------+----------+--------------------+
|        Lakeville|          1|Example Bank|  1234565276|          MN|     Bridle Court|     55044|2018-04-18T16:51:...|
|          Huntley|          2|Example Bank|  1234618993|          IL|Washington Street|     60142|2018-04-18T16:51:...|
|SouthRichmondHill|          3|Example Bank|  1234985926|          NY|    Warren Street|     11419|2018-04-18T16:51:...|
+-----------------+-----------+------------+------------+------------+-----------------+----------+--------------------+
only showing top 3 rows



In [148]:
df_branch.printSchema()

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)



In [149]:
cols=['BRANCH_CODE','BRANCH_ZIP']
for col_name in cols:
    df_branch=df_branch.withColumn(col_name,col(col_name).cast('int'))
df_branch=df_branch.withColumn('BRANCH_PHONE',regexp_replace(col('BRANCH_PHONE'),'^(\d{3})(\d{3})(\d{4})$','($1)$2-$3'))
df_branch=df_branch.withColumn('LAST_UPDATED',from_utc_timestamp('LAST_UPDATED','UTC'))

In [150]:
df_branch.show(3)

+-----------------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|       LAST_UPDATED|
+-----------------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
|        Lakeville|          1|Example Bank|(123)456-5276|          MN|     Bridle Court|     55044|2018-04-18 16:51:47|
|          Huntley|          2|Example Bank|(123)461-8993|          IL|Washington Street|     60142|2018-04-18 16:51:47|
|SouthRichmondHill|          3|Example Bank|(123)498-5926|          NY|    Warren Street|     11419|2018-04-18 16:51:47|
+-----------------+-----------+------------+-------------+------------+-----------------+----------+-------------------+
only showing top 3 rows



In [151]:
col_order=['BRANCH_CODE','BRANCH_NAME','BRANCH_STREET','BRANCH_CITY','BRANCH_STATE','BRANCH_ZIP','BRANCH_PHONE','LAST_UPDATED']
df_branch=df_branch.select(*col_order)
df_branch.show(3)

+-----------+------------+-----------------+-----------------+------------+----------+-------------+-------------------+
|BRANCH_CODE| BRANCH_NAME|    BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP| BRANCH_PHONE|       LAST_UPDATED|
+-----------+------------+-----------------+-----------------+------------+----------+-------------+-------------------+
|          1|Example Bank|     Bridle Court|        Lakeville|          MN|     55044|(123)456-5276|2018-04-18 16:51:47|
|          2|Example Bank|Washington Street|          Huntley|          IL|     60142|(123)461-8993|2018-04-18 16:51:47|
|          3|Example Bank|    Warren Street|SouthRichmondHill|          NY|     11419|(123)498-5926|2018-04-18 16:51:47|
+-----------+------------+-----------------+-----------------+------------+----------+-------------+-------------------+
only showing top 3 rows



Cleaning file 'cdw_sapp_credit.json'

In [152]:
df_credit=spark.read.json('cdw_sapp_credit.json')
check_nulls(df_credit).show()

+-----------+--------------+--------+---+-----+--------------+----------------+-----------------+----+
|BRANCH_CODE|CREDIT_CARD_NO|CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|
+-----------+--------------+--------+---+-----+--------------+----------------+-----------------+----+
|          0|             0|       0|  0|    0|             0|               0|                0|   0|
+-----------+--------------+--------+---+-----+--------------+----------------+-----------------+----+



In [153]:
df_credit.show(3)

+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|        114|4210653349028689|123459988| 14|    2|             1|       Education|             78.9|2018|
|         35|4210653349028689|123459988| 20|    3|             2|   Entertainment|            14.24|2018|
|        160|4210653349028689|123459988|  8|    7|             3|         Grocery|             56.7|2018|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
only showing top 3 rows



In [154]:
#checking for duplicates
print(df_credit.distinct().count()==df_credit.count())
df_credit.groupBy(df_credit.columns).count().where('count>1').show()

True
+-----------+--------------+--------+---+-----+--------------+----------------+-----------------+----+-----+
|BRANCH_CODE|CREDIT_CARD_NO|CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|count|
+-----------+--------------+--------+---+-----+--------------+----------------+-----------------+----+-----+
+-----------+--------------+--------+---+-----+--------------+----------------+-----------------+----+-----+



In [155]:
df_credit.printSchema()

root
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_SSN: long (nullable = true)
 |-- DAY: long (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- YEAR: long (nullable = true)



In [156]:
col_name=['CUST_SSN','BRANCH_CODE','TRANSACTION_ID']
for cols in col_name:
    df_credit=df_credit.withColumn(cols,col(cols).cast('int'))
df_credit = df_credit.withColumn("TIMEID", concat(df_credit["year"].cast("string"),
            lpad(df_credit["month"].cast("string"), 2, "0"),lpad(df_credit["day"].cast("string"), 2, "0")))
df_credit=df_credit.drop(*['DAY','MONTH','YEAR'])

In [157]:
col_order=['TRANSACTION_ID','TRANSACTION_TYPE','TRANSACTION_VALUE','CREDIT_CARD_NO','BRANCH_CODE','CUST_SSN','TIMEID']
df_credit=df_credit.select(*col_order)
df_credit.show()

+--------------+----------------+-----------------+----------------+-----------+---------+--------+
|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|  CREDIT_CARD_NO|BRANCH_CODE| CUST_SSN|  TIMEID|
+--------------+----------------+-----------------+----------------+-----------+---------+--------+
|             1|       Education|             78.9|4210653349028689|        114|123459988|20180214|
|             2|   Entertainment|            14.24|4210653349028689|         35|123459988|20180320|
|             3|         Grocery|             56.7|4210653349028689|        160|123459988|20180708|
|             4|   Entertainment|            59.73|4210653349028689|        114|123459988|20180419|
|             5|             Gas|             3.59|4210653349028689|         93|123459988|20181010|
|             6|       Education|             6.89|4210653349028689|        164|123459988|20180528|
|             7|   Entertainment|            43.39|4210653349028689|        119|123459988|20180519|


https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json

In [161]:
import requests
url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'
resp = requests.get(url).json()

In [162]:
resp

[{'Application_ID': 'LP001002',
  'Gender': 'Male',
  'Married': 'No',
  'Dependents': '0',
  'Education': 'Graduate',
  'Self_Employed': 'No',
  'Credit_History': 1,
  'Property_Area': 'Urban',
  'Income': 'medium',
  'Application_Status': 'Y'},
 {'Application_ID': 'LP001003',
  'Gender': 'Male',
  'Married': 'Yes',
  'Dependents': '1',
  'Education': 'Graduate',
  'Self_Employed': 'No',
  'Credit_History': 1,
  'Property_Area': 'Rural',
  'Income': 'medium',
  'Application_Status': 'N'},
 {'Application_ID': 'LP001005',
  'Gender': 'Male',
  'Married': 'Yes',
  'Dependents': '0',
  'Education': 'Graduate',
  'Self_Employed': 'Yes',
  'Credit_History': 1,
  'Property_Area': 'Urban',
  'Income': 'low',
  'Application_Status': 'Y'},
 {'Application_ID': 'LP001006',
  'Gender': 'Male',
  'Married': 'Yes',
  'Dependents': '0',
  'Education': 'Not Graduate',
  'Self_Employed': 'No',
  'Credit_History': 1,
  'Property_Area': 'Urban',
  'Income': 'low',
  'Application_Status': 'Y'},
 {'Applica